<a href="https://colab.research.google.com/github/joaowinderfeldbussolotto/assistente-ppc-ciencia-da-computacao/blob/main/tcc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU langchain-pinecone pinecone-notebooks openparse[ml] html2text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.2/396.2 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━

## Configuração de Variaveis

In [2]:
from google.colab import userdata

class Settings:
  HF_TOKEN          = userdata.get('HF_TOKEN')
  PINECONE_API_KEY  = userdata.get('PINECONE_API_KEY')
  GROQ_API_KEY      = userdata.get('GROQ_API_KEY')


settings = Settings()

## Pré-processamento de dados

### Download do dataset

In [3]:
!mkdir -p data
!wget https://www.uffs.edu.br/atos-normativos/ppc/ccccch/2017-0002/@@download/documento_historico -O "data/ppc_2018.pdf"

--2024-09-01 21:00:57--  https://www.uffs.edu.br/atos-normativos/ppc/ccccch/2017-0002/@@download/documento_historico
Resolving www.uffs.edu.br (www.uffs.edu.br)... 200.135.49.107
Connecting to www.uffs.edu.br (www.uffs.edu.br)|200.135.49.107|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1383621 (1.3M) [application/pdf]
Saving to: ‘data/ppc_2018.pdf’

data/ppc_2018.pdf   100%[===================>]   1.32M  1.33MB/s    in 1.0s    

2024-09-01 21:00:59 (1.33 MB/s) - ‘data/ppc_2018.pdf’ saved [1383621/1383621]



In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!cp /content/drive/My\ Drive/tcc/PPC_2024.pdf data/ppc_2024.pdf



### Transformação PDF -> HTML -> Markdown




In [7]:
import openparse
import html2text
from langchain_core.documents import Document
from uuid import uuid4


def parse_document_to_documents(file_path, source_name):
    # Parse the document using OpenParse
    parser = openparse.DocumentParser(table_args={"parsing_algorithm": "pymupdf", "table_output_format": "markdown"})
    document = parser.parse(file_path)

    documents = []
    ids = []

    # Iterate over the nodes and convert each to a Document
    for node in document.nodes:
        html_content = node.text

        if html_content:
            id = str(uuid4())
            ids.append(id)
            document_chunk = Document(
                metadata={'source': source_name, 'id': str(uuid4())},
                page_content=html_content
            )
            documents.append(document_chunk)

    return documents, ids


In [8]:
ppc_2018_chunks, ppc_2018_ids = parse_document_to_documents('data/ppc_2018.pdf', 'ppc2018')

In [9]:
ppc_2024_chunks, ppc_2024_ids = parse_document_to_documents('data/ppc_2024.pdf', 'ppc2024')

In [10]:
!pip install -qU langchain-huggingface langchain langchain-community unstructured==0.15.0 langchain_experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 46.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.3/204.3 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━

### Instanciação e download do modelo de embeddings

In [11]:
class EmbeddingModelSpecs:
  def __init__(self):
    self.name      = 'sentence-transformers/distiluse-base-multilingual-cased-v1'
    self.dimension = 512

embeddings_model = EmbeddingModelSpecs()


In [12]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name=embeddings_model.name)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

### Semantic Chunking (piorou)

In [ ]:
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(embeddings, breakpoint_threshold_type="percentile")


In [ ]:
ppc_2018_semantic_chunks = semantic_chunker.create_documents([d.page_content for d in ppc_2018_chunks])

In [ ]:
len(ppc_2018_chunks), len(ppc_2018_semantic_chunks)

(405, 866)

In [ ]:
ppc_2024_semantic_chunks = semantic_chunker.create_documents([d.page_content for d in ppc_2024_chunks])

In [ ]:
len(ppc_2024_chunks), len(ppc_2024_semantic_chunks)

(508, 948)

In [ ]:
!pip install -qU langchain-community faiss-cpu



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 60.9 MB/s eta 0:00:00


### Geração de embeddings e indíces do banco vetorial

In [13]:
import getpass
import os
import time

from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key=settings.PINECONE_API_KEY)

In [14]:
import time

index_name = "index-ppc-markdown-ids"
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=embeddings_model.dimension,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [15]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

### Ingestão de dados

In [16]:
from uuid import uuid4
from langchain_core.documents import Document
from langchain.document_loaders import UnstructuredMarkdownLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def load_documents(file_path):
    """Load documents from a Markdown file using UnstructuredMarkdownLoader."""
    # Usar um loader específico para Markdown
    loader = UnstructuredMarkdownLoader(file_path)
    documents = loader.load()
    return documents

def split_documents_into_chunks(documents, chunk_size=2000, chunk_overlap=400):
    """Split the loaded documents into text chunks based on Markdown structure."""
    # Usando RecursiveCharacterTextSplitter para respeitar a estrutura do Markdown
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n# ", "\n## ", "\n### ", "\n", " "]
    )
    texts = text_splitter.split_documents(documents)
    return texts

def create_documents_from_chunks(texts, source_name):
    """Create Document objects from text chunks."""
    documents = [
        Document(page_content=text.page_content, metadata={"source": source_name}) for text in texts
    ]
    return documents

def generate_unique_ids(num_ids):
    """Generate a list of unique UUIDs."""
    return [str(uuid4()) for _ in range(num_ids)]

def add_documents_to_vector_store(vector_store, documents, uuids):
    """Add documents with unique IDs to the vector store."""
    print(vector_store.add_documents(documents=documents, ids=uuids))


#### Ingestão PPC 2018

In [ ]:
# file_path = "data/ppc_2018.md"
# text = load_documents(file_path)
# chunks = split_documents_into_chunks(text)
# documents = create_documents_from_chunks(chunks, source_name="ppc2018.md")


In [17]:
# uuids = generate_unique_ids(len(ppc_2018_chunks))
add_documents_to_vector_store(vector_store, ppc_2018_chunks, ppc_2018_ids)

['3e41ea04-3a42-4fb3-b959-f0037b9070b4', 'd2b3f818-c60a-4256-b062-5a25906b1038', '44bd3271-8972-4d66-a8ab-1ec17e01b219', 'ff147650-0426-4b4e-9d23-1509aff0adb7', 'b81f6fe2-132b-4b1b-89a8-eadd1f5852f5', '5e250410-9cbc-43ac-81cf-47bcdd56a4a7', 'a6b933fb-91cf-40a8-b1e3-747eb63fb500', '75f55154-3630-4409-ad94-0c5a6ac93bce', 'c257832e-f380-437e-8311-ce545f796189', '1f7547bf-52ec-4bdd-9e8a-94c91737b3e1', '4ae3212d-13f7-463d-8cba-66b5a44e06fb', '1a84d530-515b-4221-b848-f66652f0f512', 'c3422b7a-8e0e-48b5-8e97-86ddeb91294b', '3cc18ecb-f824-44d4-8ea1-773516ed4115', '77df0d17-c7a7-49b8-a1df-a0c015231846', 'fcea7695-fd38-4ff2-bee1-77eec3b8f708', '042c4882-e6b7-4b13-ae8d-13d61a4b7320', 'd7a89485-4406-45aa-9b65-e604ae011996', '3a12f079-d33d-4a5f-816a-ac9cbf69bed6', 'dc33aa7c-96d2-4efd-8156-dd4646356393', 'b44bf8ba-768e-4b92-a3b5-dc5be6b65640', '1a6260ae-d385-43a9-b355-ee180eced3a8', 'e188627d-53c6-452b-afc4-1422d04b836e', '0888ad30-953a-42d3-9d7d-b823add28f77', '6596c4d6-3953-49a9-8934-510ca62504ee',

#### Ingestão PPC 2024

In [ ]:
# file_path = "data/ppc_2024.md"
# text = load_documents(file_path)
# chunks = split_documents_into_chunks(text)
# documents = create_documents_from_chunks(chunks, source_name="ppc2024.md")


In [18]:
uuids = generate_unique_ids(len(ppc_2024_chunks))
add_documents_to_vector_store(vector_store, ppc_2024_chunks, ppc_2024_ids)

['a46926ee-70ef-4c08-917c-b6dfee8a8d02', '30a66aa6-e3ef-425c-ad1f-3367240defe7', '64e7e59b-4a69-419c-878c-753d4f4e8a4a', '160aca83-d346-4417-b767-bd4e08706fe8', '33fbb48c-a828-4eb6-9733-b5820c1154b1', '1e45623f-bd91-41dd-a23e-b21c44c2e8fe', 'f521c835-53ba-41b6-94db-cae9a715cae4', '4b4c5217-dedc-433b-b5ea-aa19a2d306b8', '2e2f215c-c023-427f-8b0f-e20542177466', 'dab4ed9c-6da2-4c7d-97a7-69fe0c5508ef', 'f2f2d104-f0c5-482c-889e-3413c5fbc2c5', '6606043e-aad5-4117-927d-a0cfa9f5b476', '51ef1a3b-08e5-4899-b519-db419dabbb12', '8f2aee96-e96a-43ba-bc41-4392c7957e00', 'a378b44f-7381-4399-b81b-10a371b24b06', 'cf3c9691-c4b3-400a-89e6-1c8cf570405f', '3d9baa0a-2d7f-4bb6-9dd0-75364f0afdcb', 'e4149f23-8b65-4fa7-831f-ecef92d847bc', '4ebf451f-0dff-4adf-96b0-f1aad7daf116', 'c520ddce-7958-4e8d-8d39-8f0164bda3bf', '58f04140-1bf1-4235-9295-01c4641e065e', '85a41600-ef47-4ff4-83e0-d46ea58bf63c', '82ed61e4-e13f-4deb-ace1-ae945896e0cf', '57d0805c-4d48-4005-8e51-9159acd51b80', 'cd3f76b6-4528-440f-a04b-1c90bd06f383',

## Geração da resposta

In [19]:
!pip install -U langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 10.3 MB/s eta 0:00:00


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq



llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    groq_api_key=settings.GROQ_API_KEY,
    max_retries=2
)



def query_transformation(query):

    query_transformation_llm = ChatGroq(
        model="gemma2-9b-it",
        temperature=0,
        groq_api_key=settings.GROQ_API_KEY,
        max_retries=2
    )

    prompt = f"""Esse é minha pergunta: {query}
    Me retorne uma string que capture os itens mais essenciais da pergunta.
    Caso seja uma pergunta, transforme em algo mais conciso. Exemplo:
    "Qual a ementa de Teoria da Computação?" --> "TEORIA DA COMPUTAÇÃO | EMENTA".
    Retorne apenas a string sem nenhuma frase introdutória.
    """

    # Criando o prompt para a transformação da consulta
    prompt_template = ChatPromptTemplate.from_messages([("human", prompt)])

    # Gerando a consulta transformada
    chain = prompt_template | query_transformation_llm
    response = chain.invoke({})
    return response.content


def get_context(query, source_name = 'ppc2024.md'):

    results = vector_store.search(
        query,
        "similarity",
        k=5,
        filter={"source": source_name},
    )


    return  '@@@@\n\n\n'.join([result.page_content for result in results])




def build_prompt(contexto, pergunta):
    # Cria o prompt para o contexto do PPC
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "Você é um assistente especializado no Plano Pedagógico do Curso (PPC) de Ciência da Computação da UFFS. "
                "Sua tarefa é fornecer respostas precisas sobre qualquer questão relacionada ao PPC, utilizando o contexto fornecido para guiar sua resposta."
                "Utilize apenas o contexto informado como referencia"
                "Caso a resposta não está no contexto, informe que não sabe a resposta e recomende consultar o documento original."

                "Responda exclusivamente em portugues brasileiro, em formato markdown"
            ),
            (
            "human",
            f"Dado o contexto: {contexto}\n\nPergunta: {pergunta}. Utilize apenas o contexto fornecido para responder"),
        ]
    )
    return prompt

def get_completion(prompt):
    chain = prompt | llm

    response = chain.invoke({})
    return response


def get_answer(query, source_name):
    expanded_query = query_transformation(query) + query
    print(expanded_query)

    context = get_context(expanded_query, source_name)
    prompt = build_prompt(context, query)
    response = get_completion(prompt)

    print(response)

    return context, response.content


In [ ]:
query = "Como FUNCIONA O TCC STARTUP??"
context, answer = get_answer(query, "ppc2018")
answer

TCC STARTUP | COMO FUNCIONA 
Como FUNCIONA O TCC STARTUP??
content='O contexto fornecido não fornece informações específicas sobre o TCC Startup. Portanto, não é possível responder a essa pergunta com base neste contexto. Recomendo consultar o documento original ou outras fontes de informação relevantes para obter detalhes sobre o TCC Startup.' response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 1951, 'total_tokens': 2031, 'completion_time': 0.127878838, 'prompt_time': 0.097317579, 'queue_time': 0.0015715500000000049, 'total_time': 0.225196417}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None} id='run-c9ba4902-cc75-4a3e-ae31-d2719302cc52-0' usage_metadata={'input_tokens': 1951, 'output_tokens': 80, 'total_tokens': 2031}


'O contexto fornecido não fornece informações específicas sobre o TCC Startup. Portanto, não é possível responder a essa pergunta com base neste contexto. Recomendo consultar o documento original ou outras fontes de informação relevantes para obter detalhes sobre o TCC Startup.'

In [ ]:
context

'**REFERÊNCIAS COMPLEMENTARES**\nANTON, H.**Cálculo**. 8. ed. São Paulo: Bookman, 2007. 1 v. \nBARBOSA, J. L. M.**Geometria Euclidiana Plana**. Rio de janeiro: SBM, 2000. (Coleção \ndo Docente de Matemática). \nFLEMMING, D. M.; GONÇALVES, M. B.**Cálculo A**. 6. ed. São Paulo: Prentice Hall, \n2007. \nLEITHOLD, L.**Cálculo com geometria analítica**. 3. ed. São Paulo: HARBRA, 1994. 1 v. \nLIMA, E. L. et al*.***A Matemática do Ensino Médio**. 6. ed. Rio de Janeiro: SBM, 2000. 2 v. \n(Coleção do Docente de Matemática). \n__ _____.**A matemática do Ensino Médio.**4. ed. Rio de Janeiro: SBM, 1999. 1 v. \n(Coleção do Docente de Matemática). \nSTEWART, J.**Cálculo**. 6. ed. São Paulo: Cengage Learning, 2010. 1 v. @@@@\n\n\nANTON, H.; BIVENS, I.; DAVIS, S.**Cálculo**. 8. ed. São Paulo: Bookman, 2007. 1 v. \nAPOSTOL, T. M.**Calculus:**one-variable calculus, with an introduction to linear algebra. 2. \ned. New York: John Wiley & Sons, 1967. 1 v. \nLARSON, R.; HOSTETLER, R. P.; EDWARDS, B. H.**Cál